### **Assignment 6**
 -- Chirag Mandal (229958) & Budhaditya Mukhopadhyay (229960)


## **Part 1**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from prepare_data2 import parse_seq
import pickle
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [0]:
DIRECTORY_URL = 'https://colab.research.google.com/drive/19QD1mqnqKQ-C-TXfznrJYGjpY9uoTC2I#scrollTo=mYRlhuwuIVl7'
FILE_NAMES = ['shakespeare.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir
!python prepare_data2.py in.txt out.txt [0-9]+:[0-9]+

In [0]:




# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("out.txt.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200)) 

# a map from characters to indices
vocab = pickle.load(open("out.txt_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}
print(vocab)
print(vocab_size)

ModuleNotFoundError: ignored

In [0]:
data = tf.data.TFRecordDataset(raw_dataset)
# a map from characters to indices
vocab = pickle.load(open("out.txt_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}
print(vocab)
print(vocab_size)
# The unique characters in the file
a = sorted(set(vocab))
print ('{} unique characters'.format(len(a)))

{'I': 3, 'N': 4, 'C': 5, '8': 6, 'x': 7, '\n': 8, 's': 9, 'O': 10, 'a': 11, '5': 12, '6': 13, '-': 14, 'Z': 15, 'e': 16, 'y': 17, 'P': 18, 'n': 19, 'D': 20, 'o': 21, 'B': 22, 'G': 23, 'K': 24, 'Q': 25, ';': 26, 'Y': 27, 'v': 28, 'i': 29, 'd': 30, 'k': 31, '.': 32, 'r': 33, '0': 34, ')': 35, '3': 36, 'u': 37, 'U': 38, ' ': 39, 'S': 40, 'E': 41, "'": 42, 'z': 43, '9': 44, 'F': 45, 'm': 46, 't': 47, 'l': 48, 'q': 49, 'A': 50, '1': 51, 'L': 52, 'M': 53, '(': 54, 'f': 55, 'b': 56, 'c': 57, 'g': 58, 'W': 59, '4': 60, 'V': 61, 'h': 62, '7': 63, 'p': 64, 'j': 65, 'J': 66, ',': 67, '!': 68, '*': 69, 'T': 70, '2': 71, '?': 72, 'H': 73, 'R': 74, ':': 75, 'w': 76, '<PAD>': 0, '<S>': 1, '</S>': 2}
77
77 unique characters


In [0]:
batch_count = 0
for batch_x in data.shuffle(100000).padded_batch(128, padded_shapes=None, padding_values=None, drop_remainder=True):batch_count=batch_count+1
print("new Batch",batch_count)

new Batch 242


In [0]:
def rnn(learning_rate, seqlen, epochs):
  hidden_size = 512
  opt = tf.keras.optimizers.Adam()
  W_xh = tf.Variable((np.random.randn(vocab_size, hidden_size)).astype('float32')*0.01)
  W_hh = tf.Variable((np.random.randn(hidden_size,hidden_size)).astype('float32')*0.01)
  W_ho = tf.Variable((np.random.randn(hidden_size,vocab_size)).astype('float32')*0.01)
  b_h = tf.Variable(np.zeros((1, hidden_size), dtype= 'float32'))
  b_o = tf.Variable(np.zeros((1, vocab_size), dtype= 'float32'))
  h_init = tf.Variable((np.random.randn(1,hidden_size)).astype('float32')*0.01)
  for epoch in range(epochs):
    batch_count = 0
    for batch_x in data.shuffle(100000).batch(128, drop_remainder=True):
      print("new Batch",batch_count)
      x_seq = batch_x
      seq_loss = 0
      with tf.GradientTape() as tape:
        for i in range(1,seqlen):
          time_step = i-1
          x1 = tf.one_hot(x_seq, vocab_size)
          y = x_seq[:,time_step+1]
          y1 = tf.one_hot(y, vocab_size)
          x_seq_t = x1[:,time_step,:]
          h_init = tf.matmul(x_seq_t, W_xh)+tf.matmul(h_init, W_hh)+b_h
          h_init = tf.nn.tanh(h_init)
          logits = tf.matmul(h_init, W_ho) + b_o
          loss_char = tf.nn.softmax_cross_entropy_with_logits(labels = y1, logits = logits)
          mean = tf.reduce_mean(loss_char)
          seq_loss += mean
      grads = tape.gradient(seq_loss, [W_xh, W_hh, W_ho, b_h, b_o])
      opt.apply_gradients(zip(grads, [W_xh, W_hh, W_ho, b_h, b_o]))
      '''Wih.assign_sub(grads[0]*learning_rate)
      Whh.assign_sub(grads[1]*learning_rate)
      Wyh.assign_sub(grads[2]*learning_rate)
      bh.assign_sub(grads[3]*learning_rate)
      bo.assign_sub(grads[4]*learning_rate)'''
      print("Epoch :{}, Average error: {} ".format(epoch,seq_loss/seqlen))
      batch_count +=1
  Trained_weights = np.array([W_xh, W_hh, W_ho, b_h, b_o])
  np.save('rnn_weights', Trained_weights)

In [0]:
rnn(0.001, 150, 1)

new Batch 0


NotFoundError: ignored

In [0]:
weights = np.load('rnn_weights.npy', allow_pickle = True)
[W_xh, W_hh, W_ho, b_h, b_o] = weights
h_init = tf.Variable((np.random.randn(1,512)).astype('float32')*0.01)
char = np.zeros((1,vocab_size),dtype= 'float32')
char[0][0] = 1
finalString = ""

for i in range (1000):
  h_init = tf.matmul(char, W_xh)+tf.matmul(h_init, W_hh)+b_h
  h_init = tf.nn.tanh(h_init)
  logits = tf.matmul(h_init, W_ho)+b_o
  prob = np.exp(logits) / np.sum(np.exp(logits))             #softmax function
  idx = np.random.choice(range(vocab_size), p=prob.ravel())  #returns a list of vocab_size elements taken at random from prob into idx
  char = np.zeros((1,vocab_size),dtype= 'float32')
  char[0][idx] = 1
  op = ind_to_ch[idx]            #index to characters
  finalString += op               
print(finalString)  


## **Part 2**

In [0]:
import tensorflow as tf
import numpy as np
import os
import time
from keras.utils.data_utils import get_file
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [0]:
path_to_file = tf.keras.utils.get_file('the-king-james-bible.txt', 'https://raw.githubusercontent.com/ErikSchierboom/sentencegenerator/master/samples/the-king-james-bible.txt')
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 4332497 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿The First Book of Moses:  Called Genesis


1:1 In the beginning God created the heaven and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.


In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

75 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  '(' :   4,
  ')' :   5,
  '*' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '0' :  10,
  '1' :  11,
  '2' :  12,
  '3' :  13,
  '4' :  14,
  '5' :  15,
  '6' :  16,
  '7' :  17,
  '8' :  18,
  '9' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\ufeffThe First Bo' ---- characters mapped to int ---- > [74 42 55 52  1 28 56 65 66 67  1 24 62]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
T
h
e
 


In [0]:
#sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
sequences = char_dataset.padded_batch(seq_length+1, padded_shapes=None, padding_values=None, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeffThe First Book of Moses:  Called Genesis\n\n\n1:1 In the beginning God created the heaven and the earth'
'.\n\n1:2 And the earth was without form, and void; and darkness was upon\nthe face of the deep. And the '
'Spirit of God moved upon the face of the\nwaters.\n\n1:3 And God said, Let there be light: and there was'
' light.\n\n1:4 And God saw the light, that it was good: and God divided the light\nfrom the darkness.\n\n1'
':5 And God called the light Day, and the darkness he called Night.\nAnd the evening and the morning we'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeffThe First Book of Moses:  Called Genesis\n\n\n1:1 In the beginning God created the heaven and the eart'
Target data: 'The First Book of Moses:  Called Genesis\n\n\n1:1 In the beginning God created the heaven and the earth'


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 74 ('\ufeff')
  expected output: 42 ('T')
Step    1
  input: 42 ('T')
  expected output: 55 ('h')
Step    2
  input: 55 ('h')
  expected output: 52 ('e')
Step    3
  input: 52 ('e')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 28 ('F')


In [0]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE,padded_shapes=None, padding_values=None, drop_remainder=True)

dataset

<PaddedBatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 75) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (128, None, 256)          19200     
_________________________________________________________________
gru_3 (GRU)                  (128, None, 1024)         3938304   
_________________________________________________________________
dense_3 (Dense)              (128, None, 75)           76875     
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([68, 54, 70, 19, 65, 60, 22, 25, 57, 14, 41,  2, 54, 40, 52, 29,  8,
       74, 63,  6, 37, 26, 54, 21, 34, 62, 29, 43, 62, 38,  2, 74, 29, 23,
       17,  3, 45, 33, 32, 14, 37, 30, 48, 69,  3, 69, 25,  4, 38, 26, 19,
        6, 65,  0,  6, 67, 69,  6, 44, 58, 47, 55,  4, 20, 23, 70, 59, 33,
       22, 26, 32, 54, 30, 59, 21, 13, 11, 16, 65, 66, 12, 57,  0, 36,  9,
        9, 22, 71, 11,  4, 62, 41, 44, 46, 43, 24, 34, 30, 41, 67])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'the people stood afar off, and Moses drew near unto the\nthick darkness where God was.\n\n20:22 And the'

Next Char Predictions: 
 "ugw9rm?Cj4S!gReG-\ufeffp*ODg;LoGUoP!\ufeffGA7'WKJ4OHav'vC(PD9*r\n*tv*VkZh(:AwlK?DJgHl;316rs2j\nN..?x1(oSVYUBLHSt"


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 100, 75)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3178225


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
335/335 [==============================] - 43s 128ms/step - loss: 2.1014
Epoch 2/10
335/335 [==============================] - 44s 130ms/step - loss: 1.3492
Epoch 3/10
335/335 [==============================] - 44s 132ms/step - loss: 1.1821
Epoch 4/10
335/335 [==============================] - 44s 133ms/step - loss: 1.1125
Epoch 5/10
335/335 [==============================] - 44s 133ms/step - loss: 1.0714
Epoch 6/10
335/335 [==============================] - 44s 132ms/step - loss: 1.0417
Epoch 7/10
335/335 [==============================] - 44s 133ms/step - loss: 1.0173
Epoch 8/10
335/335 [==============================] - 44s 132ms/step - loss: 0.9973
Epoch 9/10
335/335 [==============================] - 44s 131ms/step - loss: 0.9790
Epoch 10/10
335/335 [==============================] - 44s 133ms/step - loss: 0.9629


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 256)            19200     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 75)             76875     
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"1:1"))

1:166059028 THost the place wherefore the Lord hath present thee, that the fruit of their troubles are manifest the minded shall say they.

1:19 And who art thornscording that I am ready also.

1:8 Let not the LORD and the second priest shall pierce and ning of righteousness, and I shall rear
unto thy brother's elect.

11:15 Because the Son of man be good will keep your hearts?  6:16 I will make thee the pained as a bread of life, whereof the LORD can
save upon Ethank of the children of
Israel, saying, I hold them that have the spirit
of things.

7:20 And thine account me are taken from life, take heed to the people that
from the dead I have very sorry when he is Christ's throne.

11:2 For the tradition for A
benct gain.

11:16 Now it came to pass about this wifor, saying, Thus
saith the LORD of hosts.

4:17 And I have repared from the Father, and the power and brethren, abaril, in the law of men!
from henceforth have joy about to be seen in himself, as an hendecr, neither have mercy o

# **Extra Functions**

In [0]:
tf.keras.Model.reset_states

<function tensorflow.python.keras.engine.network.Network.reset_states>

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_4 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [0]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.173309803009033
Epoch 1 Batch 100 Loss 2.347487449645996
Epoch 1 Loss 2.1414
Time taken for 1 epoch 12.885401964187622 sec

Epoch 2 Batch 0 Loss 2.1676666736602783
Epoch 2 Batch 100 Loss 1.9348502159118652
Epoch 2 Loss 1.8182
Time taken for 1 epoch 11.865055561065674 sec

Epoch 3 Batch 0 Loss 1.7781000137329102
Epoch 3 Batch 100 Loss 1.6764963865280151
Epoch 3 Loss 1.5959
Time taken for 1 epoch 12.137167692184448 sec

Epoch 4 Batch 0 Loss 1.5909942388534546
Epoch 4 Batch 100 Loss 1.4862899780273438
Epoch 4 Loss 1.5410
Time taken for 1 epoch 11.986477613449097 sec

Epoch 5 Batch 0 Loss 1.4566761255264282
Epoch 5 Batch 100 Loss 1.4397575855255127
Epoch 5 Loss 1.4653
Time taken for 1 epoch 12.101974725723267 sec

Epoch 6 Batch 0 Loss 1.4007296562194824
Epoch 6 Batch 100 Loss 1.3805246353149414
Epoch 6 Loss 1.3879
Time taken for 1 epoch 12.189754724502563 sec

Epoch 7 Batch 0 Loss 1.3064182996749878
Epoch 7 Batch 100 Loss 1.311234474182129
Epoch 7 Loss 1.3675
Time ta

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from prepare_data2 import parse_seq
import pickle
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [0]:
path_to_file = '/content/drive/My Drive/Colab Notebooks/the-king-james-bible.txt.tfrecords'
raw_dataset = tf.data.TFRecordDataset(path_to_file)
for raw_record in raw_dataset.take(10):
  print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b"\n:\n8\n\x03seq\x121\x1a/\n-\x01\x1e=\x0b5&;,@95\x1b<<\x035<\x055\x14<@\x0b@055'\n##\x0b\x045H\x0b\x0c\x0b@;@EEE\x02">
<tf.Tensor: shape=(), dtype=string, numpy=b'\nH\nF\n\x03seq\x12?\x1a=\n;\x0158\x0c59=\x0b5 \x0bC;\x0c\x0c;\x0cC5H<\x0454,\x0b\n9\x0b\x0459=\x0b5=\x0b\nA\x0b\x0c5\n\x0c\x0459=\x0b5\x0b\n,9=$EE\x02'>
<tf.Tensor: shape=(), dtype=string, numpy=b'\n\xa4\x01\n\xa1\x01\n\x03seq\x12\x99\x01\x1a\x96\x01\n\x93\x01\x015\x12\x0c\x0459=\x0b5\x0b\n,9=5\x06\n@5\x06;9=<K95\x05<,%J5\n\x0c\x045A<;\x04"5\n\x0c\x045\x04\n,\x03\x0c\x0b@@5\x06\n@5K.<\x0cE9=\x0b5\x05\n4\x0b5<\x0559=\x0b5\x04\x0b\x0b.$5\x12\x0c\x0459=\x0b5\x1f.;,;95<\x055H<\x045%<A\x0b\x045K.<\x0c59=\x0b5\x05\n4\x0b5<\x0559=\x0bE\x06\n9\x0b,@$EE\x02'>
<tf.Tensor: shape=(), dtype=string, numpy=b'\nH\nF\n\x03seq\x12?\x1a=\n;\x015\x12\x0c\x045H<\x045@\n;\x04J5\x08\x0b959=\x0b,\x0b5 \x0b5#;C=905\n\x0c\x0459=\x0b,\x0b5\x06\n@5#;C=9$EE\x02'>
<tf.Tensor: shape=(), dtype=string, numpy=b'\ng

In [0]:
#sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
sequences = data.padded_batch(seq_length+1, padded_shapes=None, padding_values=None, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

NotFoundError: ignored